In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

### load and merge data

In [ ]:
#data_path = "../data/raw" 
prod_data = pd.read_excel("../data/raw/Product_data.xlsx", sheet_name="Data")
trans_data = pd.read_excel("../data/raw/Transaction_data.xlsx", sheet_name="Data")

In [ ]:
def merge_product_transactions(product_data=prod_data, transaction_data=trans_data):
    '''
    Returns a merged dataframe of product data and transaction data.
        Parameters:
            product_data (df): A dataframe containing product information
            transaction_data (df): A dataframe containing transaction information.
        Returns:
            merged_data (df): A dataframe containing prodcut and transaction data, merged on sku.
    '''
    merged_data = pd.merge(prod_data, trans_data, on="sku", how="right")
    return merged_data

In [ ]:
merged_data = merge_product_transactions(product_data=prod_data, transaction_data=trans_data)

In [ ]:
merged_data.columns

### remove characters from the p_id_x / p_id_y

In [ ]:
def fix_pid_cols(df):
    '''
    Removes extra characters from product ID (p_id) column.
        Parameters:
            df (df): Dataframe from which to remove extra characters from p_id column.
        Returns:
            df (df): A dataframe with extra characters removed from the product ID (p_id) column.
    '''
    merged_data['p_id'] = merged_data['p_id_y'].astype('str').str.replace(r'v\d', '')
    return df

In [ ]:
merged_data = fix_pid_cols(merged_data)

In [ ]:
merged_data['p_id'].isna().value_counts()


### which sub-department has the most transactions?

In [ ]:
merged_data.groupby(['sub_department_desc']).size().sort_values(ascending=False)

In [ ]:
# how many product types exist within knits and dresses?

print('unique products in knits (p_id): ' + str(merged_data[merged_data['sub_department_desc'] == 'W L/S KNITS']['p_id'].nunique()))
print('unique products in knits (p_id_x): ' + str(merged_data[merged_data['sub_department_desc'] == 'W L/S KNITS']['p_id_x'].nunique()))
print('unique products in knits (p_id_y): ' + str(merged_data[merged_data['sub_department_desc'] == 'W L/S KNITS']['p_id_y'].nunique()))
print('unique products in knits (style): ' + str(merged_data[merged_data['sub_department_desc'] == 'W L/S KNITS']['style'].nunique()))

print('unique products in dresses (p_id): ' + str(merged_data[merged_data['sub_department_desc'] == 'DRESSES']['p_id'].nunique()))
print('unique products in dresses (p_id_x): ' + str(merged_data[merged_data['sub_department_desc'] == 'DRESSES']['p_id_x'].nunique()))
print('unique products in dresses (p_id_y): ' + str(merged_data[merged_data['sub_department_desc'] == 'DRESSES']['p_id_y'].nunique()))
print('unique products in dresses (style): ' + str(merged_data[merged_data['sub_department_desc'] == 'DRESSES']['style'].nunique()))

In [ ]:
merged_data[merged_data['sub_department_desc'] == 'W L/S KNITS'][['p_id', 'p_id_x', 'p_id_y', 'style']].drop_duplicates().sort_values('style')

### simplify colours

In [ ]:
def simplify_colours(df):
    '''
    Simplifies colours present in the 'color' column of dataframe.
        Parameters:
            df (df): Dataframe in which to simplify colours.
        Returns: 
            df (df): A dataframe containing an extra column 'color_simple' that contains simplified colours.
    '''
    df['color_simple'] = df['color']

    contains_colour = ['Windsor Heather',  'Zoe Wash', 'Woodbury Strp Wh/Hydran/G',
       'Woodburn Patchwork', 'Wooster/Alley', 'Woodley Plaid',
       'York Pld/Oxford Pld Gld', 'Watercolor Print 1', 'Vintage Sailboat',
       'Wiggins', 'Winona Wash', 'Vintage Port Multi', 'Williams Wash'
       ,'Yucatan','Rose','Gold','Wine','Navy','Royal','Wisteria','Whiskey','Coral',
       'Lavender','Tan','Khaki','Camo','Taupe','Wildflower','Floral','Hibiscus','Silver',
       'Pepper','Vicuna','Washed Forest','Mauve','Camel','Light Indigo','Whyskey','Windsor Heather Multi Str',
       'Woodbridge Olive','Yuca Tan','Yanda Wash','White','Zebra','Blue','Cream','Navy','Grey','Orange',
       'Green','Red','Rose','Mauve','Purple','Black','Pink','Brown','Yellow']

    replace_colour = ['Other','Other','Other','Other','Other','Other','Other','Other','Other','Other','Other',
        'Other','Other','Other','Pink','Yellow','Purple','Blue','Blue','Purple','Brown','Red','Purple','Brown','Green',
        'Green','Brown','Floral','Floral','Pink','Grey','Grey','Brown','Green','Purple','Brown','Blue','Brown','Windsor Heather',
        'Green','Yucatan','White','White','Zebra','Blue','Cream','Navy','Grey','Orange','Green','Red','Rose','Mauve','Purple','Black',
        'Pink','Brown','Yellow']
        
    for ii in range(len(df['color_simple'])):
        for jj in range(len(contains_colour)):
            if contains_colour[jj] in df['color_simple'][ii]: 
                df['color_simple'][ii] = replace_colour[jj]
                break
    return df

In [ ]:
merged_data = simplify_colours(merged_data)

### select columns of interest, sub-category of interest and positive transactions

In [ ]:
def select_columns_category(df):
    '''
    Creates a new dataframe that contains columns to take forward to the next step, rows matching the 'W L/S KNITS' subdepartment, 
    and rows deonoting positive transactions.
        Parameters:
            df (df): Dataframe from which to extract columns and rows.
        Returns:
            df (df): Dataframe containing columns and rows of interest.

    '''
    df = df[['p_id', 'transaction_date', 'sub_department_desc', 'label_desc', 'color_simple', 'quantity', 'amount']]
    df = df[df['sub_department_desc'] == 'W L/S KNITS']
    df = df[df['amount'] > 0]
    return(df)

In [ ]:
merged_data_pruned_sd_knits = select_columns_category(merged_data)

In [ ]:
merged_data_pruned_sd_knits.shape

### write data to interim folder

In [ ]:
merged_data_pruned_sd_knits.to_csv("../data/interim/transactions_sd_knits.csv", index=False)